In [1]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
more_features = True
use_staking = False
use_kernal_params = False
do_augment = False
fast = True

train_df = pd.read_csv('input/train' + ('_more_features' if more_features else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_more_features' if more_features else '') + '.csv')

do_lda = False

fix_data_skew = False

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [3]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [4]:
_, X_bottomhalf, _, y_bottomhalf = train_test_split(X, y, test_size=0.8, random_state=10)

In [5]:
if use_staking:
    X = X_bottomhalf
    y = y_bottomhalf

In [6]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [7]:
params = {
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_data_in_leaf': 2881,
    'max_depth': 0,
    'num_leaves': 3,
    'learning_rate': 0.01,
    'bagging_freq': 3,
    #'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 0.9),
    'feature_fraction': 0.8990901412442585,
    'bagging_seed': 11,
    'reg_alpha':  1.1173044727720816,
    'reg_lambda': 6.9285776442737514,
    'random_state': 42,
    'verbosity': -1,
    'subsample': 0.8054415526396443,
    'min_child_weight': 38.138072621096654,
    'num_threads': 4,
    'max_bin': 483
}

if use_kernal_params:
    # https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment (lb: 0.901)
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 13,
        "learning_rate" : 0.01,
        "bagging_freq": 5,
        "bagging_fraction" : 0.4,
        "feature_fraction" : 0.05,
        "min_data_in_leaf": 80,
        "min_sum_heassian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

In [8]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [9]:
oof = train_df[['ID_code', 'target']]
oof['predict'] = 0

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
predictions = test_df[['ID_code']]
val_aucs = []

In [11]:
score = 0.0
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    N = 5
    
    if not do_augment:
        N = 1
    
    p_valid,yp = 0,0
    for i in range(N):
        if do_augment:
            X_t, y_t = augment(X_train.values, y_train.values)
            X_t = pd.DataFrame(X_t)
            X_t = X_t.add_prefix('var_')

            trn_data = lgb.Dataset(X_t, label=y_t)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        else:
            trn_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        
        evals_result = {}
        
        early_stopping_rounds = 800 if fast else 3000
        
        lgb_clf = lgb.train(params,
                        trn_data,
                        num_boost_round=2000000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=early_stopping_rounds,
                        verbose_eval = 300,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    
    oof['predict'][valid_index] = p_valid/N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    predictions['fold{}'.format(fold_n+1)] = yp/N

Fold 0 started at Wed Mar 20 19:52:04 2019
Training until validation scores don't improve for 800 rounds.
[300]	training's auc: 0.764666	valid_1's auc: 0.756397
[600]	training's auc: 0.803243	valid_1's auc: 0.796463
[900]	training's auc: 0.824374	valid_1's auc: 0.817086
[1200]	training's auc: 0.838272	valid_1's auc: 0.830219
[1500]	training's auc: 0.848728	valid_1's auc: 0.83991
[1800]	training's auc: 0.85703	valid_1's auc: 0.847752
[2100]	training's auc: 0.863006	valid_1's auc: 0.853462
[2400]	training's auc: 0.868626	valid_1's auc: 0.858622
[2700]	training's auc: 0.87324	valid_1's auc: 0.862825
[3000]	training's auc: 0.877158	valid_1's auc: 0.86625
[3300]	training's auc: 0.880551	valid_1's auc: 0.869255
[3600]	training's auc: 0.883544	valid_1's auc: 0.87207
[3900]	training's auc: 0.886399	valid_1's auc: 0.874392
[4200]	training's auc: 0.888776	valid_1's auc: 0.876419
[4500]	training's auc: 0.890881	valid_1's auc: 0.878338
[4800]	training's auc: 0.892805	valid_1's auc: 0.879963
[5100]

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

Fold 1 started at Wed Mar 20 20:18:08 2019
Training until validation scores don't improve for 800 rounds.
[300]	training's auc: 0.767058	valid_1's auc: 0.762215
[600]	training's auc: 0.80511	valid_1's auc: 0.798732
[900]	training's auc: 0.824532	valid_1's auc: 0.817533
[1200]	training's auc: 0.839067	valid_1's auc: 0.83151
[1500]	training's auc: 0.84911	valid_1's auc: 0.841434
[1800]	training's auc: 0.857136	valid_1's auc: 0.849155
[2100]	training's auc: 0.863629	valid_1's auc: 0.855183
[2400]	training's auc: 0.868864	valid_1's auc: 0.860013
[2700]	training's auc: 0.873209	valid_1's auc: 0.863955
[3000]	training's auc: 0.877144	valid_1's auc: 0.867619
[3300]	training's auc: 0.880526	valid_1's auc: 0.870808
[3600]	training's auc: 0.883547	valid_1's auc: 0.873336
[3900]	training's auc: 0.88604	valid_1's auc: 0.875477
[4200]	training's auc: 0.888391	valid_1's auc: 0.877522
[4500]	training's auc: 0.890639	valid_1's auc: 0.879545
[4800]	training's auc: 0.892521	valid_1's auc: 0.881019
[5100

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

Fold 2 started at Wed Mar 20 20:46:42 2019
Training until validation scores don't improve for 800 rounds.
[300]	training's auc: 0.765824	valid_1's auc: 0.761076
[600]	training's auc: 0.804028	valid_1's auc: 0.798571
[900]	training's auc: 0.824212	valid_1's auc: 0.818489
[1200]	training's auc: 0.837563	valid_1's auc: 0.831862
[1500]	training's auc: 0.847927	valid_1's auc: 0.842026
[1800]	training's auc: 0.856143	valid_1's auc: 0.849753
[2100]	training's auc: 0.862409	valid_1's auc: 0.855875
[2400]	training's auc: 0.867789	valid_1's auc: 0.861017
[2700]	training's auc: 0.872268	valid_1's auc: 0.865508
[3000]	training's auc: 0.876035	valid_1's auc: 0.869167
[3300]	training's auc: 0.879281	valid_1's auc: 0.872432
[3600]	training's auc: 0.8822	valid_1's auc: 0.875193
[3900]	training's auc: 0.884693	valid_1's auc: 0.877663
[4200]	training's auc: 0.887172	valid_1's auc: 0.880134
[4500]	training's auc: 0.889213	valid_1's auc: 0.882129
[4800]	training's auc: 0.891069	valid_1's auc: 0.883808
[51

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

Fold 3 started at Wed Mar 20 21:12:43 2019
Training until validation scores don't improve for 800 rounds.
[300]	training's auc: 0.769496	valid_1's auc: 0.753755
[600]	training's auc: 0.803329	valid_1's auc: 0.788742
[900]	training's auc: 0.823603	valid_1's auc: 0.80949
[1200]	training's auc: 0.837699	valid_1's auc: 0.823539
[1500]	training's auc: 0.848329	valid_1's auc: 0.834405
[1800]	training's auc: 0.856249	valid_1's auc: 0.842636
[2100]	training's auc: 0.8631	valid_1's auc: 0.849074
[2400]	training's auc: 0.868373	valid_1's auc: 0.854376
[2700]	training's auc: 0.873037	valid_1's auc: 0.858885
[3000]	training's auc: 0.876968	valid_1's auc: 0.862473
[3300]	training's auc: 0.88015	valid_1's auc: 0.865655
[3600]	training's auc: 0.883483	valid_1's auc: 0.868906
[3900]	training's auc: 0.886116	valid_1's auc: 0.871347
[4200]	training's auc: 0.888532	valid_1's auc: 0.873524
[4500]	training's auc: 0.890766	valid_1's auc: 0.875424
[4800]	training's auc: 0.892606	valid_1's auc: 0.877096
[5100

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

Fold 4 started at Wed Mar 20 21:43:58 2019
Training until validation scores don't improve for 800 rounds.
[300]	training's auc: 0.769202	valid_1's auc: 0.763071
[600]	training's auc: 0.805126	valid_1's auc: 0.795851
[900]	training's auc: 0.82644	valid_1's auc: 0.815057
[1200]	training's auc: 0.840805	valid_1's auc: 0.827596
[1500]	training's auc: 0.850529	valid_1's auc: 0.836147
[1800]	training's auc: 0.859006	valid_1's auc: 0.843409
[2100]	training's auc: 0.865264	valid_1's auc: 0.849164
[2400]	training's auc: 0.870568	valid_1's auc: 0.853942
[2700]	training's auc: 0.874948	valid_1's auc: 0.857959
[3000]	training's auc: 0.878794	valid_1's auc: 0.861251
[3300]	training's auc: 0.882029	valid_1's auc: 0.864285
[3600]	training's auc: 0.884869	valid_1's auc: 0.866817
[3900]	training's auc: 0.88752	valid_1's auc: 0.869282
[4200]	training's auc: 0.889757	valid_1's auc: 0.871293
[4500]	training's auc: 0.891543	valid_1's auc: 0.872975
[4800]	training's auc: 0.893309	valid_1's auc: 0.874567
[51

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

In [12]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(oof['target'], oof['predict'])
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.900231796, std: 0.003011367. All auc: 0.900165222.


In [13]:
# submission
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('lgb_all_predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = predictions['target']
sub_df.to_csv("lgb_submission.csv", index=False)
oof.to_csv('lgb_oof.csv', index=False)

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
